In [9]:
from ovito.io import import_file, export_file
from ovito.modifiers import CalculateDisplacementsModifier
from ovito.modifiers import ExpressionSelectionModifier
from pymatgen.core import Structure
import numpy as np

strc = Structure.from_file('super.cif')
ref = Structure.from_file('prim_inverse_noli.cif')

def round_to_value(a):
    multiplier = a/0.125
    rounded_multiplier = np.round(multiplier, 0)
    return rounded_multiplier * 0.125

def reorder(a):
    position = [0, 1, 2]
    for i in position:
        if a[i] == 1.:
            a[i] = 0

ref.make_supercell([3,3,3])

for atom in strc:
    strc_corrected = (round_to_value(atom.frac_coords))
    atom.frac_coords = strc_corrected
    reorder(atom.frac_coords)

mobile_sites = []

for atom1 in strc:
    for atom2 in ref:
        if np.array_equal((atom1.frac_coords), (atom2.frac_coords), equal_nan=False) == True:
            if atom2.species_string == 'Na':
                print(f'{atom1.frac_coords} is 8b')
                mobile_sites.append(atom1)
            elif atom2.species_string == 'Ca':
                print(f'{atom1.frac_coords} is 8a')
                mobile_sites.append(atom1)
            elif atom2.species_string == 'Be':
                print(f'{atom1.frac_coords} is 16c')
                mobile_sites.append(atom1)
            elif atom2.species_string == 'Nb':
                print(f'{atom1.frac_coords} is 48f')
                mobile_sites.append(atom1)

mobile_indices = [i for i, site in enumerate(list(mobile_sites))] 



[0.125 0.125 0.625] is 8a
[0.875 0.625 0.375] is 48f
[0.625 0.125 0.125] is 8a
[0.875 0.625 0.375] is 48f
[0.125 0.125 0.125] is 8b
[0.125 0.125 0.125] is 8b
[0.625 0.125 0.625] is 8b
[0.125 0.125 0.125] is 8b
[0.625 0.625 0.125] is 8b
[0.625 0.625 0.125] is 8b
[0.625 0.625 0.875] is 48f
[0.625 0.625 0.125] is 8b
[0.875 0.625 0.375] is 48f
[0.625 0.125 0.625] is 8b
[0.125 0.125 0.125] is 8b
[0.625 0.125 0.625] is 8b
[0.625 0.125 0.625] is 8b
[0.875 0.625 0.375] is 48f
[0.625 0.625 0.125] is 8b
[0.625 0.625 0.875] is 48f
[0.125 0.125 0.125] is 8b
[0.625 0.125 0.125] is 8a
[0.625 0.625 0.875] is 48f
[0.125 0.125 0.125] is 8b
[0.625 0.125 0.125] is 8a
[0.625 0.125 0.625] is 8b
[0.625 0.625 0.875] is 48f
[0.625 0.125 0.625] is 8b
[0.125 0.125 0.125] is 8b
[0.875 0.625 0.375] is 48f
[0.625 0.125 0.125] is 8a
[0.625 0.125 0.625] is 8b
[0.125 0.125 0.125] is 8b
[0.875 0.625 0.375] is 48f
[0.125 0.125 0.625] is 8a
[0.625 0.625 0.125] is 8b
[0.125 0.125 0.625] is 8a
[0.625 0.625 0.125] is 8b
[0

In [11]:
mobile_indices

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47]

In [ ]:
# Load input data and create a data pipeline.
pipeline = import_file("super_al.lammpstrj")

# Calculate per-particle displacements with respect to initial simulation frame:
pipeline.modifiers.append(CalculateDisplacementsModifier())

# Define the custom modifier function:
def calculate_msd(frame, data):

    # Access the per-particle displacement magnitudes computed by the 
    # CalculateDisplacementsModifier that precedes this user-defined modifier in the 
    # data pipeline:
    displacement_magnitudes = data.particles['Displacement Magnitude']
    
    mobile_displacement_magnitudes = []
    
    for ind in mobile_indices:
        mobile_displacement_magnitudes.append(displacement_magnitudes[ind])
    
    # Compute MSD:
    msd = numpy.sum(mobile_displacement_magnitudes ** 2) / len(mobile_displacement_magnitudes)

    # Output MSD value as a global attribute: 
    data.attributes["MSD"] = msd 
    
# Insert user-defined modifier function into the data pipeline.
pipeline.modifiers.append(calculate_msd)

# Export calculated MSD value to a text file and let OVITO's data pipeline do the rest:
export_file(pipeline, "mobile_li_600.txt", 
    format = "txt/attr",
    columns = ["Timestep", "MSD"],
    multiple_frames = True)